In [36]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
    
#From my EEG package 
import run_expts
import constants


#Let me see as many results as I want to see
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 20)

In [4]:
load_path = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/Data/Catch_22_features/'

In [172]:
X = pd.read_hdf(load_path + 'REM_c_22_feautures.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'REM_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'REM_groups.h5', key='df', mode='r')

#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 4 )

# # #One channel per region is worse than three channels 
# X_cols = X_expt.columns
# one_chan_per_reg = [channels[1] for channels in reg_chs]
# selected_cols = [col for col in X_cols if any(col.startswith(prefix) for prefix in one_chan_per_reg)]
# X_expt = X_expt[selected_cols]

X_expt = X_expt[ [col for col in X_expt.columns if 'Fpz' == col[:3] or 'FCz'== col[:3] or 'POz'== col[:3] ] ]

# X_expt = X_expt[ [col for col in X_expt.columns if chan == col[:len(chan)] for chan in channels ]]

In [182]:
X_expt

,Fpz_DN_HistogramMode_5,Fpz_DN_HistogramMode_10,Fpz_SB_BinaryStats_diff_longstretch0,Fpz_DN_OutlierInclude_p_001_mdrmd,Fpz_DN_OutlierInclude_n_001_mdrmd,Fpz_CO_f1ecac,Fpz_CO_FirstMin_ac,Fpz_SP_Summaries_welch_rect_area_5_1,Fpz_SP_Summaries_welch_rect_centroid,Fpz_FC_LocalSimple_mean3_stderr,...,POz_SB_BinaryStats_mean_longstretch1,POz_SB_MotifThree_quantile_hh,POz_FC_LocalSimple_mean1_tauresrat,POz_CO_Embed2_Dist_tau_d_expfit_meandiff,POz_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,POz_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,POz_SB_TransitionMatrix_3ac_sumdiagcov,POz_PD_PeriodicityWang_th0_01,POz_StandardDeviation,POz_Mean
0,-2.841434,0.313940,272.0,0.032390,0.034711,12.0,20.0,21.322590,0.086562,2.272537,...,28.0,1.560258,0.024390,0.090393,0.42,0.50,0.000073,33.0,5.880081,-0.001143
1,1.978857,-0.908866,259.0,-0.002346,-0.001628,12.0,20.0,18.462780,0.080935,2.106292,...,31.0,1.567569,0.018750,0.081298,0.48,0.54,0.000065,31.0,5.509858,-0.003174
2,0.331202,-3.973876,392.0,0.003075,0.002171,38.0,331.0,35.347343,0.031809,1.572912,...,54.0,1.394977,0.060606,0.036330,0.36,0.42,0.000061,30.0,7.301861,0.008359
3,-3.169513,2.199171,530.0,-0.269873,-0.265292,39.0,14.0,35.556252,0.028618,2.020711,...,34.0,1.485292,0.020548,0.043262,0.40,0.48,0.000064,20.0,7.509506,-0.006630
4,-3.186679,1.752790,659.0,-0.218303,-0.216764,56.0,19.0,54.931560,0.023837,1.919861,...,39.0,1.441940,0.058333,0.056963,0.44,0.50,0.000153,25.0,9.112752,0.001066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2.979325,-2.054560,381.0,-0.073650,-0.100510,47.0,176.0,64.683069,0.023489,2.194623,...,43.0,1.372750,0.076923,0.018586,0.44,0.50,0.000083,713.0,10.135353,-0.003489
65,6.920259,2.407572,553.0,0.209845,0.187354,41.0,19.0,25.572896,0.029481,2.271046,...,25.0,1.598061,0.016304,0.048149,0.44,0.52,0.000108,23.0,5.700521,-0.002941
66,2.287960,-1.273477,440.0,0.057549,0.055784,37.0,6.0,24.312139,0.034143,2.380403,...,28.0,1.551246,0.012658,0.043561,0.48,0.52,0.000063,6.0,6.980328,-0.008517
67,-0.478103,3.863650,327.0,-0.139259,-0.141356,25.0,174.0,28.214765,0.045756,2.015482,...,36.0,1.471177,0.026667,0.021297,0.42,0.48,0.000063,22.0,8.089097,-0.026754


In [174]:
len(selected_cols)

312

In [175]:
reg_chs=['Fp1', 'Fpz', 'Fp2'], ['F1', 'Fz', 'F2'], ['AF7', 'F7', 'F5', 'F3'], ['AF8', 'F8', 'F6', 'F4'], \
            ['FCz', 'FC1', 'FC2', 'Cz', 'C1', 'C2', 'CP1', 'CP2'], ['FC5', 'FC3', 'C5', 'C3', 'CP5', 'CP3'], \
            ['FC6', 'FC4', 'C6', 'C4', 'CP6', 'CP4'], ['TP7', 'T7', 'FT7', 'FT9'], ['TP8', 'T8', 'FT8', 'FT10'], \
            ['P1', 'P2', 'Pz'], ['P3', 'P5', 'P7', 'PO7'], ['P4', 'P6', 'P8', 'PO8'], ['POz', 'O1', 'O2', 'Oz']

In [176]:
one_chan_per_reg = [channels[1] for channels in reg_chs]
len(one_chan_per_reg)

13

In [177]:
X_train

,Fpz_DN_HistogramMode_5,Fpz_DN_HistogramMode_10,Fpz_SB_BinaryStats_diff_longstretch0,Fpz_DN_OutlierInclude_p_001_mdrmd,Fpz_DN_OutlierInclude_n_001_mdrmd,Fpz_CO_f1ecac,Fpz_CO_FirstMin_ac,Fpz_SP_Summaries_welch_rect_area_5_1,Fpz_SP_Summaries_welch_rect_centroid,Fpz_FC_LocalSimple_mean3_stderr,...,POz_SB_BinaryStats_mean_longstretch1,POz_SB_MotifThree_quantile_hh,POz_FC_LocalSimple_mean1_tauresrat,POz_CO_Embed2_Dist_tau_d_expfit_meandiff,POz_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,POz_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,POz_SB_TransitionMatrix_3ac_sumdiagcov,POz_PD_PeriodicityWang_th0_01,POz_StandardDeviation,POz_Mean
29,-8.413773,-4.211666,598.0,-0.028491,-0.038218,76.0,224.0,21.115482,0.014249,1.426877,...,40.0,1.416447,0.015625,0.019193,0.44,0.50,0.000021,962.0,6.420435,-0.023111
48,11.498175,5.310123,444.0,-0.121118,-0.109851,40.0,19.0,29.963096,0.032945,2.257276,...,31.0,1.514523,0.022857,0.034658,0.38,0.46,0.000023,21.0,5.357168,-0.003584
43,-3.792192,16.961145,827.0,-0.109033,-0.078512,73.0,15.0,36.536371,0.019193,1.819250,...,28.0,1.701973,0.015267,0.021534,0.42,0.48,0.000041,4.0,7.087514,-0.004932
66,2.287960,-1.273477,440.0,0.057549,0.055784,37.0,6.0,24.312139,0.034143,2.380403,...,28.0,1.551246,0.012658,0.043561,0.48,0.52,0.000063,6.0,6.980328,-0.008517
51,4.720513,1.141798,618.0,-0.216889,-0.240627,91.0,45.0,27.357542,0.014705,1.560013,...,34.0,1.451196,0.020513,0.050867,0.46,0.50,0.000036,41.0,4.602373,0.003172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,3.598721,-1.356228,530.0,-0.017387,-0.018373,54.0,13.0,28.598081,0.023297,1.986251,...,31.0,1.551423,0.015228,0.042949,0.40,0.48,0.000161,14.0,6.783279,-0.019655
34,1.017698,-3.981320,200.0,-0.071536,-0.074089,21.0,54.0,21.983412,0.054576,2.173572,...,27.0,1.530151,0.023077,0.059123,0.50,0.56,0.000114,30.0,7.192500,-0.005935
15,-6.693948,-0.228414,638.0,0.067757,0.073359,108.0,490.0,28.163089,0.009659,1.242912,...,35.0,1.394954,0.016529,0.018126,0.46,0.50,0.000067,1070.0,7.472770,0.010278
53,7.748096,3.076350,538.0,-0.115642,-0.133902,51.0,17.0,29.124274,0.025044,1.566524,...,37.0,1.441372,0.058824,0.042667,0.44,0.46,0.000016,25.0,7.420304,0.000804


In [183]:
acc_scores = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_expt , y_expt, test_size=0.2)
    clf = RandomForestClassifier()

    t1 = time.time()

    clf.fit(X_train, y_train)
    y_preds = clf.predict(X_test)
    
    acc = accuracy_score(y_preds , y_test )
    

    print(acc )
    
    acc_scores.append(acc)

    t2 = time.time()

mean_acc = np.mean(np.array(acc_scores))
print('mean accuracy is:' + str(mean_acc) )
#     print(t2-t1)

0.35714285714285715
0.42857142857142855
0.35714285714285715
0.35714285714285715
0.5
0.35714285714285715
0.35714285714285715
0.42857142857142855
0.35714285714285715
0.5
mean accuracy is:0.4


In [105]:
# Best mean is around 70 with features from three different channels